In [62]:
import pandas as pd
import numpy as np
import math
import warnings
# from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore')

In [63]:
#Import
amesdfTest = pd.read_csv('../data/test.csv', index_col=0)
amesdfTrain = pd.read_csv('../data/train.csv', index_col=0)

In [64]:
#Import the CSV to make DF, count all NA values in the DF
amesdfTest['SalePrice'] = 0
amesDF = pd.concat([amesdfTrain, amesdfTest], axis=0)
amesDFcols = np.array(amesDF.columns)
amesDFnas = []
for i in range(len(amesDFcols)):
    amesDFnas.append(amesDF[amesDFcols[i]].isna().sum())
amesDFsummarydf = pd.DataFrame({'varnames': amesDFcols, 'nas': amesDFnas})

In [65]:
#Print the DF in descending order of number of NA values
amesDFsummarydf.sort_values(by='nas', ascending=False)

,varnames,nas
71,PoolQC,2909
73,MiscFeature,2814
5,Alley,2721
72,Fence,2348
56,FireplaceQu,1420
2,LotFrontage,486
63,GarageCond,159
62,GarageQual,159
58,GarageYrBlt,159
59,GarageFinish,159


In [66]:
#For most columns (as seen from the documentation), NA does not mean NA but that the house does not have that feature
#These aren't actually NAs, as they contain information
#We can replace NA with the word 'No', to indicate the feature doesn't exist for this house
#Define function to do this for multiple column names
def ConvertNaToNone(df, varlist):
    amescopy = df.copy()
    for var in varlist:
        series = amescopy[var]
        series[series.isna()] = 'No'
        amescopy[var] = series
    return amescopy 

In [67]:
#Apply this function with our list of features containing NA, and convert the "No" in MasVnrType to "None" to match the other values
amesNoNAs = ConvertNaToNone(amesDF, ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageYrBlt', 'GarageCond', 'BsmtExposure', 'BsmtFinType2', 'GarageType', 'GarageFinish', 'GarageQual', 'BsmtCond', 'BsmtQual', 'BsmtFinType1', 'MasVnrType'])
amesNoNAs['MasVnrType'] = amesNoNAs['MasVnrType'].str.replace('None', 'No')

In [68]:
#Now, count NAs for the remaining features as we did above
amesNoNAcols = np.array(amesNoNAs.columns)
amesNoNAnas = []
for i in range(len(amesNoNAcols)):
    amesNoNAnas.append(amesNoNAs[amesNoNAcols[i]].isna().sum())
amesNoNAsummarydf = pd.DataFrame({'varnames': amesNoNAcols, 'nas': amesNoNAnas})
amesNoNAsummarydf.sort_values(by='nas', ascending=False)

,varnames,nas
2,LotFrontage,486
25,MasVnrArea,23
1,MSZoning,4
47,BsmtHalfBath,2
8,Utilities,2
54,Functional,2
46,BsmtFullBath,2
61,GarageArea,1
41,Electrical,1
37,TotalBsmtSF,1


In [69]:
#For LotFrontage, replace with random sample of the non-NA values, for MasVnrArea, impute with 0:
amesNoNAs.loc[amesNoNAs['LotFrontage'].isna(), 'LotFrontage'] = np.random.choice(amesNoNAs['LotFrontage'].dropna(), size=486, replace=True)
amesNoNAs.loc[amesNoNAs['MasVnrArea'].isna(), 'MasVnrArea'] = np.zeros(23)

In [70]:
#In the test DF, there are extra Bsmt and Garage columns with NA, that should be 0 (verified). Replace with "No" for now (will convert to 0 later)
amesNoNAs = ConvertNaToNone(amesNoNAs, ['BsmtFullBath', 'BsmtHalfBath', 'GarageArea', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF1', 'BsmtFinSF2', 'GarageCars'])

In [71]:
#Check NA's again:
amesNoNAcols = np.array(amesNoNAs.columns)
amesNoNAnas = []
for i in range(len(amesNoNAcols)):
    amesNoNAnas.append(amesNoNAs[amesNoNAcols[i]].isna().sum())
amesNoNAsummarydf = pd.DataFrame({'varnames': amesNoNAcols, 'nas': amesNoNAnas})
amesNoNAsummarydf.sort_values(by='nas', ascending=False)

,varnames,nas
1,MSZoning,4
8,Utilities,2
54,Functional,2
41,Electrical,1
52,KitchenQual,1
22,Exterior1st,1
23,Exterior2nd,1
77,SaleType,1
47,BsmtHalfBath,0
46,BsmtFullBath,0


In [72]:
#We have a few missing categoricals here. Simply impute with the msot common value:
print(amesNoNAs['Electrical'].value_counts())
print(amesNoNAs['MSZoning'].value_counts())
print(amesNoNAs['Utilities'].value_counts())
print(amesNoNAs['Functional'].value_counts())
print(amesNoNAs['Exterior2nd'].value_counts())
print(amesNoNAs['Exterior1st'].value_counts())
print(amesNoNAs['KitchenQual'].value_counts())
print(amesNoNAs['SaleType'].value_counts())

SBrkr    2671
FuseA     188
FuseF      50
FuseP       8
Mix         1
Name: Electrical, dtype: int64
RL         2265
RM          460
FV          139
RH           26
C (all)      25
Name: MSZoning, dtype: int64
AllPub    2916
NoSeWa       1
Name: Utilities, dtype: int64
Typ     2717
Min2      70
Min1      65
Mod       35
Maj1      19
Maj2       9
Sev        2
Name: Functional, dtype: int64
VinylSd    1014
MetalSd     447
HdBoard     406
Wd Sdng     391
Plywood     270
CmentBd     126
Wd Shng      81
Stucco       47
BrkFace      47
AsbShng      38
Brk Cmn      22
ImStucc      15
Stone         6
AsphShn       4
CBlock        3
Other         1
Name: Exterior2nd, dtype: int64
VinylSd    1025
MetalSd     450
HdBoard     442
Wd Sdng     411
Plywood     221
CemntBd     126
BrkFace      87
WdShing      56
AsbShng      44
Stucco       43
BrkComm       6
CBlock        2
AsphShn       2
Stone         2
ImStucc       1
Name: Exterior1st, dtype: int64
TA    1492
Gd    1151
Ex     205
Fa      70
Name

In [12]:
#Now impute with these top values:
amesNoNAs.loc[amesNoNAs['Electrical'].isna(),'Electrical'] = 'SBrkr'
amesNoNAs.loc[amesNoNAs['MSZoning'].isna(),'MSZoning'] = 'RL'
amesNoNAs.loc[amesNoNAs['Utilities'].isna(),'Utilities'] = 'AllPub'
amesNoNAs.loc[amesNoNAs['Functional'].isna(),'Functional'] = 'Typ'
amesNoNAs.loc[amesNoNAs['Exterior1st'].isna(),'Exterior1st'] = 'VinylSd'
amesNoNAs.loc[amesNoNAs['Exterior2nd'].isna(),'Exterior2nd'] = 'VinylSd'
amesNoNAs.loc[amesNoNAs['KitchenQual'].isna(),'KitchenQual'] = 'TA'
amesNoNAs.loc[amesNoNAs['SaleType'].isna(),'SaleType'] = 'WD'

In [73]:
#Sum all the row of the Series counting NA's in each column (total NAs in dataframe) to show all NA's are gone
np.sum(np.sum(amesNoNAs.isna()))

13

In [74]:
#Remove MSSubClass (basically a copy of house type and bldg type and year):
amesNoNAs = amesNoNAs.drop('MSSubClass', axis=1)

In [75]:
#Check Zoning and rewrite C (all) as C:
print(amesNoNAs['MSZoning'].value_counts())
amesNoNAs['MSZoning'] = amesNoNAs['MSZoning'].replace('C (all)', 'C')

RL         2265
RM          460
FV          139
RH           26
C (all)      25
Name: MSZoning, dtype: int64


In [76]:
#Rewrite LotShape as numeric:
amesNoNAs['LotShape'] = amesNoNAs['LotShape'].replace(['Reg', 'IR1', 'IR2', 'IR3'], ['0','1','2','3'])

In [77]:
#Change Utilities to Number of Utilities available:
amesNoNAs['Utilities'] = amesNoNAs['Utilities'].replace(['AllPub', 'NoSewr', 'NoSeWa', 'ELO'], [4,3,2,1])

In [78]:
#Change LandSlope to numeric:
amesNoNAs['LandSlope'] = amesNoNAs['LandSlope'].replace(['Gtl', 'Mod', 'Sev'], [1,2,3])

In [79]:
#Drop age of garage, as this is a troublesome metric (what to do with garage-less homes?)
amesNoNAs = amesNoNAs.drop('GarageYrBlt', axis=1)

In [80]:
#Drop GarageCars, as this metric is too correlated to GarageArea, which gives more detail:
amesNoNAs = amesNoNAs.drop('GarageCars', axis=1)

In [81]:
#Finish, Quality, and Condition are all similar metrics for a garage, which will add too much variance if considered separately. Let's average them together and make one metric:
amesNoNAs['GarageQual'] = amesNoNAs['GarageQual'].replace(['No', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], [0,1,2,3,4,5])
amesNoNAs['GarageCond'] = amesNoNAs['GarageCond'].replace(['No', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], [0,1,2,3,4,5])
amesNoNAs['GarageFinish'] = amesNoNAs['GarageFinish'].replace(['No', 'Unf', 'RFn', 'Fin'], [0,1,2,3])


In [82]:
amesNoNAs['GarageScore'] = (amesNoNAs['GarageQual'] + amesNoNAs['GarageCond'] + amesNoNAs['GarageFinish'])/3

In [83]:
amesNoNAs = amesNoNAs.drop(['GarageQual', 'GarageCond', 'GarageFinish'], axis=1)

In [84]:
#A 3-season porch sounds a lot like an enclosed porch, and there are very few of them. It would simplify the analysis if we considered a 3ssn porch as an enclosed porch:
amesNoNAs['EnclosedPorch'] = amesNoNAs['EnclosedPorch'] + amesNoNAs['3SsnPorch']
amesNoNAs = amesNoNAs.drop('3SsnPorch', axis=1)

In [85]:
#We have only 12 Minimum Wood fence. It would seem intuitive that a minimal fence of either type (wood/vinyl) may have similar value, let's try combining them:
amesNoNAs['Fence'].value_counts()
amesNoNAs['Fence'] = amesNoNAs['Fence'].replace(['MnPrv', 'MnWw'], ['Minimal', 'Minimal'])

In [86]:
#By the descriptions, ConLD, ConLI, ConLW, and Con sound similar. Converting 3 variables to 1 will also reduce noise:
print(amesNoNAs['SaleType'].value_counts())
amesNoNAs['SaleType'] = amesNoNAs['SaleType'].replace(['ConLD', 'ConLI', 'ConLw', 'Con'], ['Contract', 'Contract', 'Contract', 'Contract'])


WD       2525
New       239
COD        87
ConLD      26
CWD        12
ConLI       9
ConLw       8
Oth         7
Con         5
Name: SaleType, dtype: int64


In [87]:
#Name the categorical variables to use to make dummies:
Categorical = ['MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig', 'Neighborhood',
                  'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Foundation', 'MasVnrType', 'Heating', 'CentralAir', 'Electrical',
                  'GarageType', 'SaleType','SaleCondition']

In [88]:
#Create BsmtScore from 0-6, described in the initial processing file:

amesNoNAs['BsmtFinType1'] = amesNoNAs['BsmtFinType1'].astype('str').copy()
amesNoNAs['BsmtFinType1'] = pd.to_numeric(amesNoNAs['BsmtFinType1'].replace(['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'No'], 
                                                              [6,5,4,3,2,1,0]))

amesNoNAs['BsmtFinType2'] = amesNoNAs['BsmtFinType2'].astype('str').copy()
amesNoNAs['BsmtFinType2'] = pd.to_numeric(amesNoNAs['BsmtFinType2'].replace(['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'No'], 
                                                              [6,5,4,3,2,1,0]))
amesNoNAs[['BsmtFinType1', 'BsmtFinType2', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF']].dtypes

amesNoNAs['BsmtFinSF1'] =  amesNoNAs['BsmtFinSF1'].replace(0, 1)
amesNoNAs['BsmtFinSF2'] =  amesNoNAs['BsmtFinSF2'].replace(0, 1)
amesNoNAs['TotalBsmtSFTemp'] =  amesNoNAs['TotalBsmtSF'].replace(0, 1)

#Convert "No" to 0 for FinSF:
amesNoNAs['BsmtFinSF1'] = amesNoNAs['BsmtFinSF1'].replace('No', 1)
amesNoNAs['BsmtFinSF2'] = amesNoNAs['BsmtFinSF2'].replace('No', 1)

amesNoNAs['BsmtScore'] = (amesNoNAs['BsmtFinType1'].values*amesNoNAs['BsmtFinSF1'].values + 
                       amesNoNAs['BsmtFinType2'].values*amesNoNAs['BsmtFinSF2'].values) / (amesNoNAs['BsmtFinSF1'].values + amesNoNAs['BsmtFinSF2'].values)

In [89]:
#Check everything is between 0 and 6
amesNoNAs['BsmtScore'].describe()

count    2919.000000
mean        3.537939
std         2.084213
min         0.000000
25%         1.000000
50%         3.991597
75%         5.918033
max         5.999114
Name: BsmtScore, dtype: float64

In [90]:
#Remove BsmtFin1,2 BsmtFinSF1,2, BsmtUnfSF, TotalBsmtSFTemp
amesNoNAs = amesNoNAs.drop(['BsmtFinType1', 'BsmtFinType2', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSFTemp'], axis=1)

In [91]:
#Make new column, TotalSF, and remove 1stFlrSF and 2ndFlrSF
amesNoNAs['TotalSF'] = amesNoNAs['1stFlrSF'].values + amesNoNAs['2ndFlrSF'].values
amesNoNAs = amesNoNAs.drop(['1stFlrSF', '2ndFlrSF'], axis=1)

In [92]:
#Check that 5 columns have been removed
print(amesNoNAs.shape)

(2919, 69)


In [93]:
#Convert FullBath, HalfBath, BsmtFullBath, BsmtHalfBath into TotalBath

#Convert the "No"s (from NA removal) to 0's:
amesNoNAs['BsmtFullBath'] = amesNoNAs['BsmtFullBath'].replace('No', 0)
amesNoNAs['BsmtHalfBath'] = amesNoNAs['BsmtHalfBath'].replace('No', 0)

amesNoNAs['TotalBath'] = amesNoNAs['FullBath'].values + amesNoNAs['BsmtFullBath'].values + 0.5*amesNoNAs['HalfBath'].values + 0.5*amesNoNAs['BsmtHalfBath'].values
amesNoNAs = amesNoNAs.drop(['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath'], axis=1)
print(amesNoNAs.shape)

(2919, 66)


In [94]:
'''Reading the data description and looking at the numbers. We essentially have 1sfFlrSF and 2ndFlrSF (TotalSF), 
and LowQualSF. GrLivArea is the sum of all three (every single time). So it can be eliminated'''

amesNoNAs = amesNoNAs.drop(['GrLivArea'], axis=1)

In [95]:
#Remove MiscFeature (because we have the corresponding "MiscValue"):
amesNoNAs = amesNoNAs.drop('MiscFeature', axis=1)
print(amesNoNAs.shape)

(2919, 64)


In [96]:
#Now convert the ordinal categorical values to integers:
AmesDummiesOrdinal = amesNoNAs.copy()
AmesDummiesOrdinal['ExterQual'] = AmesDummiesOrdinal['ExterQual'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], ['1','2','3','4','5'])
AmesDummiesOrdinal['ExterCond'] = AmesDummiesOrdinal['ExterCond'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], ['1','2','3','4','5'])
AmesDummiesOrdinal['BsmtQual'] = AmesDummiesOrdinal['BsmtQual'].replace(['No', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], ['0','1','2','3','4','5'])
AmesDummiesOrdinal['BsmtCond'] = AmesDummiesOrdinal['BsmtCond'].replace(['No', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], ['0','1','2','3','4','5'])
AmesDummiesOrdinal['BsmtExposure'] = AmesDummiesOrdinal['BsmtExposure'].replace(['No', 'Mn', 'Av', 'Gd'], ['0','1','2','3'])
AmesDummiesOrdinal['HeatingQC'] = AmesDummiesOrdinal['HeatingQC'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], ['1','2','3','4','5'])
AmesDummiesOrdinal['KitchenQual'] = AmesDummiesOrdinal['KitchenQual'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], ['1','2','3','4','5'])
AmesDummiesOrdinal['FireplaceQu'] = AmesDummiesOrdinal['FireplaceQu'].replace(['No', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], ['0','1','2','3','4','5'])
AmesDummiesOrdinal['PoolQC'] = AmesDummiesOrdinal['PoolQC'].replace(['No', 'Fa', 'TA', 'Gd', 'Ex'], ['0','1','2','3','4'])
AmesDummiesOrdinal['Functional'] = AmesDummiesOrdinal['Functional'].replace(['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'], ['1','2','3','4','5', '6', '7', '8'])
AmesDummiesOrdinal['PavedDrive'] = AmesDummiesOrdinal['PavedDrive'].replace(['N', 'P', 'Y'], ['0','1','2'])


In [97]:
#Define function to replace all categorical variables (from a list) with dummies in a given Df:
def ReplaceWithDummies(df, dummylist):
    df2 = df.copy()
    for var in dummylist:
        topindex = df2[var].value_counts().sort_values(ascending=False).index[0]
        dummies = pd.get_dummies(df2[var], prefix=var)
        dummies = dummies.drop(var + "_" + str(topindex), axis=1)
        df2 = pd.concat([df2, dummies], axis=1)
        df2 = df2.drop(var, axis=1)
    return df2

In [98]:
#Conver the pure categorical values to Dummies:
AmesDummiesOrdinal = ReplaceWithDummies(AmesDummiesOrdinal, Categorical)
print(AmesDummiesOrdinal.shape)

(2919, 141)


In [99]:
#Also do this for Fence:
AmesDummiesOrdinal = ReplaceWithDummies(AmesDummiesOrdinal, ['Fence'])

In [100]:
#Check out value counts for the two Joint categories:
print(amesNoNAs['Condition1'].value_counts())
print(amesNoNAs['Condition2'].value_counts())
print(amesNoNAs['Exterior1st'].value_counts())
print(amesNoNAs['Exterior2nd'].value_counts())

Norm      2511
Feedr      164
Artery      92
RRAn        50
PosN        39
RRAe        28
PosA        20
RRNn         9
RRNe         6
Name: Condition1, dtype: int64
Norm      2889
Feedr       13
Artery       5
PosN         4
PosA         4
RRNn         2
RRAe         1
RRAn         1
Name: Condition2, dtype: int64
VinylSd    1025
MetalSd     450
HdBoard     442
Wd Sdng     411
Plywood     221
CemntBd     126
BrkFace      87
WdShing      56
AsbShng      44
Stucco       43
BrkComm       6
CBlock        2
AsphShn       2
Stone         2
ImStucc       1
Name: Exterior1st, dtype: int64
VinylSd    1014
MetalSd     447
HdBoard     406
Wd Sdng     391
Plywood     270
CmentBd     126
Wd Shng      81
Stucco       47
BrkFace      47
AsbShng      38
Brk Cmn      22
ImStucc      15
Stone         6
AsphShn       4
CBlock        3
Other         1
Name: Exterior2nd, dtype: int64


In [101]:
#Make Dummy DF adding these together
Cond1DF = pd.get_dummies(amesNoNAs['Condition1'], prefix='Condition1')
Cond2DF = pd.get_dummies(amesNoNAs['Condition2'], prefix='Condition2')
Ext1DF = pd.get_dummies(amesNoNAs['Exterior1st'], prefix='Exterior1')
Ext2DF = pd.get_dummies(amesNoNAs['Exterior2nd'], prefix='Exterior2')
Joint = pd.concat([Cond1DF, Cond2DF, Ext1DF, Ext2DF], axis=1)
Joint.head()

'''There is a typographical error in the Kaggle data! For variable Exterior1st, there is a CemntBd entry, and for
Exterior2nd, there is a CmentBd entry. These are clearly the same, so let's fix them.

We have a similar problem for WdShing and Wd Shng and BrkComm/Brk Cmn'''

Joint['Exterior2_CemntBd'] = Joint['Exterior2_CmentBd'].copy()
Joint = Joint.drop('Exterior2_CmentBd', axis=1)

Joint['Exterior2_WdShing'] = Joint['Exterior2_Wd Shng'].copy()
Joint = Joint.drop('Exterior2_Wd Shng', axis=1)

Joint['Exterior2_BrkComm'] = Joint['Exterior2_Brk Cmn'].copy()
Joint = Joint.drop('Exterior2_Brk Cmn', axis=1)


#Now, combine 1 and 2 columns and drop each individual component:
for var in amesNoNAs['Condition1'].value_counts().index:
    if var in ["RRNe"]:
        continue
    Joint['Condition_' + var] = Joint['Condition1_' + var] + Joint['Condition2_' + var]
    Joint = Joint.drop('Condition1_' + var, axis=1)
    Joint = Joint.drop('Condition2_' + var, axis=1)
    
for var in amesNoNAs['Exterior1st'].value_counts().index:
    Joint['Exterior_' + var] = Joint['Exterior1_' + var] + Joint['Exterior2_' + var]
    Joint = Joint.drop('Exterior1_' + var, axis=1)
    Joint = Joint.drop('Exterior2_' + var, axis=1)  

In [102]:
#Check Joint to see how it looks:
Joint.head()

,Condition1_RRNe,Exterior2_Other,Condition_Norm,Condition_Feedr,Condition_Artery,Condition_RRAn,Condition_PosN,Condition_RRAe,Condition_PosA,Condition_RRNn,...,Exterior_CemntBd,Exterior_BrkFace,Exterior_WdShing,Exterior_AsbShng,Exterior_Stucco,Exterior_BrkComm,Exterior_CBlock,Exterior_AsphShn,Exterior_Stone,Exterior_ImStucc
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,2,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
#We have 6 columns to rename (values not present in one or the other 1st/2nd):
Joint['Condition_RRNe'] = Joint['Condition1_RRNe'].copy()
Joint = Joint.drop('Condition1_RRNe', axis=1)

Joint['Exterior_Other'] = Joint['Exterior2_Other'].copy()
Joint = Joint.drop('Exterior2_Other', axis=1)

#Drop the most common values:
Joint = Joint.drop('Condition_Norm', axis=1)
Joint = Joint.drop('Exterior_VinylSd', axis=1)

for var in Joint.columns:
    Joint[var] = Joint[var].replace(2, 1)
    
Joint.head()

,Condition_Feedr,Condition_Artery,Condition_RRAn,Condition_PosN,Condition_RRAe,Condition_PosA,Condition_RRNn,Exterior_MetalSd,Exterior_HdBoard,Exterior_Wd Sdng,...,Exterior_WdShing,Exterior_AsbShng,Exterior_Stucco,Exterior_BrkComm,Exterior_CBlock,Exterior_AsphShn,Exterior_Stone,Exterior_ImStucc,Condition_RRNe,Exterior_Other
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
#Living Near a Park is good, living near train tracks is bad according to most common perception. We can convert PosN, PosA, RRAe, RRAn, RRNn, RRNe to a ProximityBonus score, based on this:
Joint['ProximityBonus'] = Joint['Condition_PosN'] + 2*Joint['Condition_PosA'] - Joint['Condition_RRNn'] - Joint['Condition_RRNe'] - 2*Joint['Condition_RRAn'] - 2*Joint['Condition_RRAe']
Joint = Joint.drop(['Condition_PosN', 'Condition_PosA', 'Condition_RRNn', 'Condition_RRNe', 'Condition_RRAn', 'Condition_RRAe'], axis=1)

In [105]:
#This all looks good -- now we can combine this DF with amesNoNAsDummies:

AmesDummiesOrdinal = pd.concat([AmesDummiesOrdinal, Joint], axis=1)
AmesDummiesOrdinal = AmesDummiesOrdinal.drop(['Condition1', 'Condition2', 'Exterior1st', 'Exterior2nd'], axis=1)
print(AmesDummiesOrdinal.shape)

(2919, 157)


In [106]:
#Convert any "No" remaining in the DF (from NA removal) to 0, because we have no more categorical variables:

for var in AmesDummiesOrdinal:
    AmesDummiesOrdinal[var] = AmesDummiesOrdinal[var].replace("No", 0)   

In [107]:
#Finally, let's make sure everything in the DF is numerical and convert any string integers to numeric:
for var in AmesDummiesOrdinal:
    AmesDummiesOrdinal[var] = pd.to_numeric(AmesDummiesOrdinal[var])

In [108]:
#Convert year numbers to "Years since..." numbers (2010 will be 1, 1999 will be 12, etc):
YearCategories = ['YearBuilt', 'YearRemodAdd','YrSold']
    
for var in YearCategories:
    AmesDummiesOrdinal[var] = 2011-AmesDummiesOrdinal[var]

In [109]:
#Rename the columns

AmesDummiesOrdinal['YearsAgoBuilt'] = AmesDummiesOrdinal['YearBuilt'].copy()
AmesDummiesOrdinal['YearsSinceRemodel'] = AmesDummiesOrdinal['YearRemodAdd'].copy()
AmesDummiesOrdinal['YearsSinceSale'] = AmesDummiesOrdinal['YrSold'].copy()

AmesDummiesOrdinal = AmesDummiesOrdinal.drop(['YearBuilt', 'YearRemodAdd','YrSold'], axis=1)

In [110]:
#Convert SaleMonth (1-12) into Winter (12,1,2), Sping(3,4,5), Summer(6,7,8), and Autumn (9,10,11)

AmesDummiesOrdinal['MoSold'] = AmesDummiesOrdinal['MoSold'].replace([12,1,2,3,4,5,6,7,8,9,10,11], ['Winter', 'Winter', 'Winter', 'Spring', 'Spring', 'Spring', 'Summer', 'Summer',
                                                  'Summer', 'Autumn', 'Autumn', 'Autumn'])

AmesDummiesOrdinal = ReplaceWithDummies(AmesDummiesOrdinal, ['MoSold'])

In [111]:
#Check the shape, compared to the Train ordinal DF (186 columns):
AmesDummiesOrdinal.shape

(2919, 159)

In [112]:
#Finally, remove the two 
AmesDummiesOrdinal = AmesDummiesOrdinal.drop([524,1299], axis=0)

In [113]:
#Export the Final Train DF:
AmesDummiesOrdinal.to_csv('AmesOrdinalManualFeatures.csv')

In [55]:
#Import the DF with the correct 62 feautures:
AmesDummiesOrdinalF = pd.read_csv('AmesOrdinalManualAICFinal.csv', index_col=0)

In [56]:
#Take the feature list (minus saleprice) from the train DF:
FeatureList = list(AmesDummiesOrdinalF.columns)
FeatureList
len(FeatureList)

62

In [57]:
#Try to create final Test/Train DF, by subsetting the columns from FeatureList:
AmesTestDFFinal = AmesDummiesOrdinal.loc[1461:]
AmesTrainDFFinal = AmesDummiesOrdinal.loc[:1461]

In [58]:
AmesTestDFFinal = AmesTestDFFinal[FeatureList]
AmesTrainDFFinal = AmesTrainDFFinal[FeatureList]

In [59]:
AmesTestDFFinal = AmesTestDFFinal.drop('SalePrice', axis=1)

In [196]:
#Write it to csv:
#AmesTestDFFinal.to_csv('AmesTestDFFinal.csv')

In [60]:
#Set up the Training model again using the train set:
#1. Import the functions:

AmesDF = AmesTrainDFFinal

from sklearn.linear_model import LinearRegression
lm = LinearRegression()

from sklearn import ensemble
randomForest = ensemble.RandomForestRegressor()
randomForest.set_params(bootstrap=False, max_depth=None, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, 
                        n_estimators=200)

from sklearn import ensemble
gradientBoost = ensemble.GradientBoostingRegressor()
gradientBoost.set_params(n_estimators=800, min_samples_split=6, min_samples_leaf=4, max_features='log2', max_depth=2, loss='huber')


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='huber', max_depth=2,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=4, min_samples_split=6,
             min_weight_fraction_leaf=0.0, n_estimators=800,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [61]:
#Rename AmesDF to X and Y
X = AmesDF.drop('SalePrice', axis=1)
Y = AmesDF['SalePrice']

#Run Linear Model:
lm.fit(X, Y)
lmpredict = lm.predict(AmesTestDFFinal)

#Run Random Forest model:
randomForest.fit(X, Y)
rfpredict = randomForest.predict(AmesTestDFFinal)

#Run Gradient Boost model:
gradientBoost.fit(X, Y)
gbpredict = gradientBoost.predict(AmesTestDFFinal)

In [222]:
#Create the final "stacked" model with 25% Linear, 25% Random Forest, 50% Gradient Boost:

finalpredictions = (40*gbpredict + 25*lmpredict + 35*rfpredict)/100

In [223]:
#Create a DF to match the Kaggle submission criteria:
SubmissionDC1116 = pd.DataFrame({'Id':AmesTestDFFinal.index, 'SalePrice':finalpredictions})

In [224]:
#Check head:
SubmissionDC1116.head()

,Id,SalePrice
0,1461,79356.688843
1,1462,163382.659524
2,1463,180292.220829
3,1464,188981.966053
4,1465,197064.898076


In [225]:
#Write to CSV:
SubmissionDC1116.to_csv('Ames_PredictionDC1116.csv', index=False)